In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:


rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [ ]:
def n_step_SARSA(gridworld:GridWorld_v2.GridWorld_v2,sarsaStep=5,epsilon=0.5,trajectorySteps=-1, learning_rate=0.001, final_epsilon=0.01, num_episodes=600, gamma = 0.9)->GridWorld_v2.GridWorld_v2:
    """
    这是n-step-SARSA算法

    Parameters:
    sarsaStep (int): 对应n-step-sarsa那个n的大小，相对于sarsa,额外多的
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    final_epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数
    gamma (float): 近视远视程度

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))  # 初始化状态价值函数为全0
    action_value = np.zeros((rows * columns, 5))  # 初始化动作价值函数为全0，5表示5种可能的动作
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))]  # 随机初始化策略，使用独热编码表示
    for episode in range(num_episodes):  # 循环迭代指定次数
        #清除输出，可以更好的展示策略
        # time.sleep(0.2)
        # clear_output(wait=True)
        
        print("episode",f"{episode}/{num_episodes}")  # 打印当前迭代次数和总迭代次数
        if(epsilon > final_epsilon) :  # 如果当前epsilon大于最终epsilon值
            epsilon -= 0.001  # 逐渐减小epsilon值
        else:
            epsilon = final_epsilon  # 否则保持epsilon为最终值

        # p1是目标方向的概率，p0是另外四个方向的概率
        p1 = 1-epsilon * (4/5)  # 计算选择最优动作的概率
        p0 = epsilon/5  # 计算选择其他动作的概率
        d = {1:p1, 0:p0}  # 创建概率字典，用于向量化操作
        # policy_epsilon是policy取epsilon-greedy的概率决策
        print("p1",p1,"p0",p0)  # 打印两种概率值
        policy_epsilon = np.vectorize(d.get)(policy)  # 将策略转换为epsilon-greedy概率策略

        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]  # 初始化访问计数器，25是状态总数
        
        initState=10  # 设置初始状态
        initAction=random.randint(0,4)  # 随机选择初始动作

        if trajectorySteps==-1:  # 如果轨迹步数为-1
            stop_when_reach_target = True  # 设置到达目标时停止
        Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                  action=initAction, 
                                                  policy=policy_epsilon, # 传入epsilon-greedy策略
                                                  steps=trajectorySteps, 
                                                  stop_when_reach_target=True)  # 获取轨迹和得分
        for i in range(sarsaStep):  # 在轨迹末尾添加额外的步骤
            Trajectory.append((17,4,1,17,4))  # 让智能体在最后自转一下，保证轨迹长度足够
        print("trajectorySteps",len(Trajectory))  # 打印轨迹长度
        

        
        # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
        
        steps = len(Trajectory)  # 获取 轨迹实际长度+sarsaStep
        # if steps>trajectorySteps:  # 如果轨迹长度超过设定值5000
        #     continue  # 跳过当前迭代
        g = 1  # 初始化折扣因子
        reward = 0  # 初始化累计奖励
        for k in range(0, steps, 1):  # 遍历整个轨迹
            tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]  # 解包轨迹中的元素
            reward += tmpscore * g  # 累加折扣后的奖励
            g *= gamma  # 更新折扣因子
        print("Trajectory_score", reward)  # 打印整个轨迹的累计奖励


        
        for k in range(0, steps - sarsaStep + 1, 1):  # 遍历轨迹，为每个状态计算n步SARSA更新
            # 记录当前状态的访问次数，Trajectory[k][0] 表示当前轨迹中第 k 个状态
            cnt[Trajectory[k][0]] += 1  # 增加当前状态的访问计数
            # n-step-SARSA 算法核心部分，计算 n 步后的回报并更新动作价值函数
            # 初始化当前状态和动作，-1 表示还未赋值
            nowState,nowAction = -1,-1  # 初始化当前状态和动作
            # 折扣因子的幂次，初始为 1
            g = 1  # 初始化折扣因子，这就是gamma的n次方
            # 累计奖励，初始为 0
            reward = 0  # 初始化累计奖励
            for i in range(sarsaStep):  # 计算n步回报
                # 从轨迹中获取当前步骤的状态、动作、奖励、下一个状态和下一个动作
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k+i]  # 解包轨迹中的元素
                # 如果 nowState 还未赋值，则将当前步骤的状态和动作赋值给它
                if nowState == -1:  # 如果是第一步
                    nowState, nowAction = tmpstate, tmpaction  # 记录初始状态和动作
                # 累加奖励，乘以对应的折扣因子
                reward += tmpscore * g  # 累加折扣后的奖励
                # 更新折扣因子的幂次
                g *= gamma  # 更新折扣因子
            # 加上 n 步后状态的动作价值函数乘以折扣因子
            reward += g*action_value[nextState][nextAction]  # 加上n步后状态的估计值
            
            # 计算时间差分（TD）误差，即当前动作价值函数与 n 步回报的差值
            TD_error = action_value[nowState][nowAction] - reward  # 计算TD误差
            # 根据 TD 误差和学习率更新动作价值函数
            action_value[nowState][nowAction] -= learning_rate * TD_error  # 更新动作价值函数

        # for k in range(steps - sarsaStep+1, steps, 1):
        #     # print(k)
        #     #State，Action，Reward，NextState，NextAction
        #     tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
        #     cnt[tmpstate] += 1
        #     TD_error = action_value[tmpstate][tmpaction] - (tmpscore + gamma * action_value[nextState][nextAction])
        #     action_value[tmpstate][tmpaction] -= learning_rate * TD_error


        # policy improvement
        policy = np.eye(5)[np.argmax(action_value,axis=1)]  # 策略改进：选择价值最高的动作作为新策略
        policy_epsilon = np.vectorize(d.get)(policy)  # 将策略转换为epsilon-greedy概率策略
    
        #输出每个state的访问次数
        print(np.array(cnt).reshape(5,5))  # 打印状态访问次数矩阵

        state_value = np.sum(policy_epsilon * action_value,axis=1)  # 计算状态价值函数
        mean_state_value = np.sum(policy_epsilon * action_value,axis=1).mean()  # 计算平均状态价值
        
        gridworld.showPolicy(policy)  # 显示当前策略
        print(np.round(action_value,decimals=3).reshape(-1,5))  # 打印动作价值函数（四舍五入到3位小数）
        print(np.round(state_value,decimals=4).reshape(5,5))  # 打印状态价值函数（四舍五入到4位小数）
        print("mean_state_value", mean_state_value)  # 打印平均状态价值

    return gridworld  # 返回更新后的网格世界

In [4]:
n_step_SARSA(gridworld,sarsaStep = 3, trajectorySteps=5000,final_epsilon=0.01,num_episodes = 600,gamma = 0.99)

episode 0/600
p1 0.6008 p0 0.0998
trajectorySteps 305
Trajectory_score -87.53326805875916
[[ 2 12 47 30  3]
 [10 23  8 29  9]
 [ 4 20  1 10 48]
 [ 0  2  2  3 32]
 [ 0  0  0  0  8]]
⬆️⬅️⬆️➡️⬆️
⬇️⏪⏬⬇️⬆️
⬇️➡️⏬➡️⬆️
⬆️⏩️✅⏫️⬇️
⬆️⏫️⬆️⬆️➡️
[[ 0.    -0.     0.    -0.001  0.   ]
 [-0.003 -0.002 -0.01   0.    -0.   ]
 [-0.008 -0.01  -0.03  -0.012 -0.065]
 [-0.002  0.    -0.03  -0.001 -0.001]
 [ 0.     0.     0.     0.     0.   ]
 [-0.001 -0.03   0.    -0.011 -0.012]
 [-0.002 -0.05  -0.129  0.     0.   ]
 [-0.001 -0.001  0.    -0.079  0.   ]
 [-0.013 -0.004  0.    -0.05  -0.   ]
 [ 0.    -0.001 -0.006 -0.    -0.   ]
 [-0.011 -0.03   0.     0.     0.   ]
 [-0.277  0.    -0.04  -0.02  -0.01 ]
 [ 0.     0.     0.002  0.     0.   ]
 [-0.02  -0.    -0.01  -0.009 -0.02 ]
 [-0.001 -0.069 -0.001 -0.019 -0.006]
 [ 0.     0.     0.     0.     0.   ]
 [-0.039  0.     0.     0.     0.   ]
 [ 0.     0.     0.002  0.     0.003]
 [ 0.    -0.001  0.     0.     0.   ]
 [-0.015 -0.002 -0.001 -0.02  -0.043]
 [ 0.   

In [5]:
n_step_SARSA(gridworld,sarsaStep = 2, trajectorySteps=5000) #变的更好一些

episode 0/600
p1 0.6008 p0 0.0998
trajectorySteps 7
Trajectory_score -6.249069000000001
[[0 0 0 0 0]
 [0 0 0 0 0]
 [1 0 0 0 0]
 [2 1 2 0 0]
 [0 0 0 0 0]]
⬆️⬆️⬆️⬆️⬆️
⬆️⏫️⏫️⬆️⬆️
⬆️⬆️⏫️⬆️⬆️
⬆️⏩️✅⏫️⬆️
⬆️⏫️⬆️⬆️⬆️
[[ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.    -0.009  0.     0.    -0.009]
 [ 0.     0.001  0.     0.     0.   ]
 [ 0.     0.     0.001  0.     0.002]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0

In [6]:
g = 1
res = []
for i in range(15):
    g *= 0.9
    res.append(g*100//1/100)
print(res)

[0.9, 0.81, 0.72, 0.65, 0.59, 0.53, 0.47, 0.43, 0.38, 0.34, 0.31, 0.28, 0.25, 0.22, 0.2]


In [7]:
g = 1
res = []
for i in range(15):
    g *= 0.99
    res.append(g*100//1/100)
print(res)

[0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.89, 0.88, 0.87, 0.86, 0.86]
